# Установка зависимостей

In [36]:
# 0. Обновление pip и инструментов сборки
%pip install -U pip setuptools wheel

# 1. Установка PyTorch (CPU-версия)
%pip install torch==2.4.1 torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

# 2. Обновление typing-extensions
%pip install -U typing-extensions

# 3. Установка зависимостей PyTorch Geometric
%pip install torch-scatter torch-sparse torch-cluster torch-spline-conv
%pip install torch-geometric==2.2.0

# 4. Установка утилит
%pip install -U scikit-learn pandas networkx matplotlib rich tqdm
%pip install graphlime focal-loss-torch stella tensorflow

3302.78s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Note: you may need to restart the kernel to use updated packages.


3309.38s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Looking in indexes: https://download.pytorch.org/whl/cpu
Note: you may need to restart the kernel to use updated packages.


3315.18s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


...

Note: you may need to restart the kernel to use updated packages.


3334.44s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Note: you may need to restart the kernel to use updated packages.


3341.40s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


  Using cached stella-0.1.0.tar.gz (13.0 MB)
  Preparing metadata (setup.py) ... done
  Using cached astropy-5.2.2-cp38-cp38-macosx_11_0_arm64.whl.metadata (8.2 kB)
  Using cached astroquery-0.4.7-py3-none-any.whl.metadata (7.2 kB)
  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package us

In [1]:
import os, random, numpy as np
import datetime
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, recall_score, confusion_matrix
import torch.optim as optim
from torch_geometric.utils import add_self_loops
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve, roc_curve, auc
import networkx as nx
from torch_geometric.explain.algorithm.gnn_explainer import GNNExplainer_
import joblib
from collections import OrderedDict
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import stellargraph as sg
from stellargraph.mapper import FullBatchNodeGenerator
from stellargraph.layer import DirectedGraphSAGE
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy
from stellargraph.mapper import DirectedGraphSAGENodeGenerator
from stellargraph.layer import DirectedGraphSAGE
from stellargraph.mapper import GraphSAGENodeGenerator
from stellargraph.layer import GraphSAGE
from stellargraph.layer.gcn import GCN
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

# Загрузка датасета

In [2]:
#@title 📂 Загрузка датасета (идемпотентно)
import os

# Папка, куда будет сохранён репозиторий
data_dir = 'data/'

import os
import shutil

data_dir = 'data/'

# Проверяем, существует ли папка
if os.path.exists(data_dir):
    # Если папка существует, удаляем все её содержимое
    for root, dirs, files in os.walk(data_dir, topdown=False):
        for name in files:
            os.remove(os.path.join(root, name))
        for name in dirs:
            os.rmdir(os.path.join(root, name))
else:
    # Если папка не существует, создаем её
    os.makedirs(data_dir)


# Клонирование репозитория в папку data
!git clone https://github.com/salam-ammari/Labeled-Transactions-based-Dataset-of-Ethereum-Network.git {data_dir}


import zipfile, pathlib
zip_path = "data/Dataset.zip"
extract_path = pathlib.Path("data/unpacked")
with zipfile.ZipFile(zip_path) as zf: zf.extractall(extract_path)
print("✅ Распаковка завершена")

Cloning into 'data'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 75 (delta 35), reused 4 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (75/75), 31.54 MiB | 9.02 MiB/s, done.
Resolving deltas: 100% (35/35), done.
✅ Распаковка завершена


In [22]:
#@title 🧹 Чтение CSV
import pandas as pd
dataset_csv_path = extract_path / "Dataset" / "Dataset.csv"
df = pd.read_csv(dataset_csv_path)
print(df.shape, "строк")
print(df.columns.tolist())

(71250, 18) строк
['hash', 'nonce', 'transaction_index', 'from_address', 'to_address', 'value', 'gas', 'gas_price', 'input', 'receipt_cumulative_gas_used', 'receipt_gas_used', 'block_timestamp', 'block_number', 'block_hash', 'from_scam', 'to_scam', 'from_category', 'to_category']


# Инженерия данных

In [23]:
# %% [data scrubbing]
# Список столбцов, в которых НЕ хотим допускать NaN
cols_required = [col for col in df.columns 
                 if col not in ['from_category', 'to_category']]

# Считаем пропуски по этим столбцам
missing_counts_req = df[cols_required].isnull().sum()
print("Missing values in required columns:\n", missing_counts_req)

# Дропаем строки, где есть хоть одно NaN в обязательных столбцах
df = df.dropna(subset=cols_required)
print("After dropping rows with missing in required cols:", df.shape)

# Проверка и дроп дубликатов
dup_count = df.duplicated().sum()
print("Duplicate rows count:", dup_count)
if dup_count > 0:
    df = df.drop_duplicates()
    print("Dropped duplicate rows. New shape:", df.shape)
else:
    print("No duplicate rows found.")

Missing values in required columns:
 hash                           0
nonce                          0
transaction_index              0
from_address                   0
to_address                     0
value                          0
gas                            0
gas_price                      0
input                          0
receipt_cumulative_gas_used    0
receipt_gas_used               0
block_timestamp                0
block_number                   0
block_hash                     0
from_scam                      0
to_scam                        0
dtype: int64
After dropping rows with missing in required cols: (71250, 18)
Duplicate rows count: 11
Dropped duplicate rows. New shape: (71239, 18)


In [24]:
# Считаем количество 0 и 1 в from_scam / to_scam
from_counts = df['from_scam'].value_counts().rename({0: 'non_scam', 1: 'scam'})
to_counts   = df['to_scam'  ].value_counts().rename({0: 'non_scam', 1: 'scam'})

print("Sender (from_scam):")
print(from_counts)
print("\nRecipient (to_scam):")
print(to_counts)

# Если вас интересуют транзакции, где есть хоть одна пометка scam:
df['any_scam'] = ((df['from_scam'] == 1) | (df['to_scam'] == 1)).astype(int)
any_counts = df['any_scam'].value_counts().rename({0: 'no_scam', 1: 'has_scam'})
print("\nTransactions with any scam flag:")
print(any_counts)

Sender (from_scam):
from_scam
non_scam    68622
scam         2617
Name: count, dtype: int64

Recipient (to_scam):
to_scam
non_scam    59601
scam        11638
Name: count, dtype: int64

Transactions with any scam flag:
any_scam
no_scam     57000
has_scam    14239
Name: count, dtype: int64


In [25]:
import pandas as pd

# # Допустим, df уже загружен и очищен от NaN/дубликатов

# # 1) Балансировка по from_scam
# n_from_scam = df['from_scam'].sum()  # должно быть 2617
# df_from_pos = df[df['from_scam'] == 1]
# df_from_neg = df[df['from_scam'] == 0].sample(n=n_from_scam, random_state=42)
# df_balanced_from = pd.concat([df_from_pos, df_from_neg]) \
#                      .sample(frac=1, random_state=42) \
#                      .reset_index(drop=True)

# print("Balanced from_scam:")
# print(df_balanced_from['from_scam'].value_counts())

# # 2) Балансировка по to_scam
# n_to_scam = df['to_scam'].sum()  # 11638
# df_to_pos = df[df['to_scam'] == 1]
# df_to_neg = df[df['to_scam'] == 0].sample(n=n_to_scam, random_state=42)
# df_balanced_to = pd.concat([df_to_pos, df_to_neg]) \
#                    .sample(frac=1, random_state=42) \
#                    .reset_index(drop=True)

# print("\nBalanced to_scam:")
# print(df_balanced_to['to_scam'].value_counts())

# Если нужно ровно 2617 для обоих (независимо от фактического числа):
# desired_n = 2617

# # По from_scam
# df_from_pos = df[df['from_scam'] == 1]
# df_from_neg = df[df['from_scam'] == 0].sample(n=desired_n, random_state=42)
# df_sampled_from_exact = pd.concat([df_from_pos, df_from_neg]) \
#                            .sample(frac=1, random_state=42) \
#                            .reset_index(drop=True)

# # По to_scam
# df_to_pos = df[df['to_scam'] == 1].sample(n=desired_n, random_state=42)
# df_to_neg = df[df['to_scam'] == 0].sample(n=desired_n, random_state=42)
# df_sampled_to_exact = pd.concat([df_to_pos, df_to_neg]) \
#                          .sample(frac=1, random_state=42) \
#                          .reset_index(drop=True)

# print("\nExact 2617 each for from_scam:")
# print(df_sampled_from_exact['from_scam'].value_counts())
# print("\nExact 2617 each for to_scam:")
# print(df_sampled_to_exact['to_scam'].value_counts())


# Предполагаем, что df уже загружен и очищен от NaN/дубликатов
desired_n = 2617

# 1) Баланс по from_scam
pos_from = df[df["from_scam"] == 1]
neg_from = df[df["from_scam"] == 0].sample(n=desired_n, random_state=42)

df_from_balanced = (
    pd.concat([pos_from, neg_from])
      .sample(frac=1, random_state=42)
      .reset_index(drop=True)
)

print("FROM balance:\n", df_from_balanced["from_scam"].value_counts())

# 2) Баланс по to_scam без пересечений с df_from_balanced
pos_to = df[df["to_scam"] == 1].sample(n=desired_n, random_state=42)

# из кандидатов на negative-to исключаем уже взятые в df_from_balanced
neg_to_candidates = df[df["to_scam"] == 0]
neg_to_candidates = neg_to_candidates.loc[
    ~neg_to_candidates.index.isin(df_from_balanced.index)
]

neg_to = neg_to_candidates.sample(n=desired_n, random_state=42)

df_to_balanced = (
    pd.concat([pos_to, neg_to])
      .sample(frac=1, random_state=42)
      .reset_index(drop=True)
)

print("\nTO balance:\n", df_to_balanced["to_scam"].value_counts())

# 3) Проверка пересечений индексов между двумя наборами
intersection = set(df_from_balanced.index) & set(df_to_balanced.index)
print("\nПересечение индексов:", intersection)

# 4) Если нужен единый датасет — объединяем и удаляем дубликаты
df_balanced = (
    pd.concat([df_from_balanced, df_to_balanced])
      .drop_duplicates()
      .reset_index(drop=True)
)
print("\nИтоговый сбалансированный датасет:", df_balanced.shape)

df = df_balanced

FROM balance:
 from_scam
0    2617
1    2617
Name: count, dtype: int64

TO balance:
 to_scam
0    2617
1    2617
Name: count, dtype: int64

Пересечение индексов: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 18

In [26]:
# %% [feature engineering]
import pandas as pd

# 0) Предполагаем, что df уже загружен и очищен от NaN/дубликатов

# 1) Извлекаем год из ISO-строки block_timestamp
df['block_year'] = df['block_timestamp'].str[:4].astype(int)

# 2) Категории value (3 квантиля)
df['value_cat'] = pd.qcut(df['value'], q=3, labels=False)

# 3) Считаем node-признаки на уровне адреса
# -------------------------------------------------------------------
# Собираем все вхождения адрес → год первой транзакции
addr_years = pd.concat([
    df[['from_address', 'block_year']].rename(columns={'from_address': 'address'}),
    df[['to_address',   'block_year']].rename(columns={'to_address':   'address'}),
], ignore_index=True)
reg_year = addr_years.groupby('address')['block_year'].min().rename('reg_year')

# Собираем все вхождения адрес → максимальное значение транзакции
addr_vals = pd.concat([
    df[['from_address', 'value']].rename(columns={'from_address': 'address'}),
    df[['to_address',   'value']].rename(columns={'to_address':   'address'}),
], ignore_index=True)
max_value = addr_vals.groupby('address')['value'].max().rename('max_value')

# Склеиваем node-таблицу
node_features_df = (
    pd.concat([reg_year, max_value], axis=1)
      .reset_index()               # index → address
)
node_features_df.columns = ['address', 'reg_year', 'max_value']

# 4) Считаем edge-признаки, мерджим туда годы работы из node-таблицы
# -------------------------------------------------------------------
# Сначала добавим «from_» признаки
edge_df = df[[
    'from_address', 'to_address',
    'gas', 'gas_price',
    'receipt_cumulative_gas_used', 'receipt_gas_used',
    'block_number', 'value_cat',
    'from_scam', 'to_scam'
]].copy()

# прицепляем год регистрации и max_value, затем считаем years_in_operation
edge_df = edge_df.merge(
    node_features_df.rename(columns={
        'address': 'from_address',
        'reg_year': 'from_reg_year',
        'max_value': 'from_max_value'
    }),
    on='from_address', how='left'
)
edge_df = edge_df.merge(
    node_features_df.rename(columns={
        'address': 'to_address',
        'reg_year': 'to_reg_year',
        'max_value': 'to_max_value'
    }),
    on='to_address', how='left'
)

current_year = edge_df['block_year'].max() if 'block_year' in edge_df else df['block_year'].max()
# Но нам нужен block_year в edge_df для расчёта лет в эксплуатации:
edge_df['block_year'] = df['block_year']  # просто копируем
edge_df['from_years_in_operation'] = current_year - edge_df['from_reg_year']
edge_df['to_years_in_operation']   = current_year - edge_df['to_reg_year']

# 5) Итоговые списки признаков
edge_feature_cols = [
    'gas', 'gas_price',
    'receipt_cumulative_gas_used', 'receipt_gas_used',
    'block_number', 'value_cat',
    'from_years_in_operation', 'to_years_in_operation'
]

node_feature_cols = [
    'address',    # ID узла
    'reg_year',   # год регистрации
    'max_value',  # макс. сумма транзакций
]

response_cols = ['from_scam', 'to_scam']

# 6) Проверим, что всё на месте
print("Edge DataFrame shape:", edge_df.shape)
print("Edge feature columns:", edge_feature_cols)
print("Node DataFrame shape:", node_features_df.shape)
print("Node feature columns:", node_feature_cols)
print("Response columns:", response_cols)

# При желании можно отделить чисто две таблицы:
node_df = node_features_df.set_index('address')
edge_df = edge_df[['from_address','to_address'] + edge_feature_cols + response_cols]

Edge DataFrame shape: (10138, 17)
Edge feature columns: ['gas', 'gas_price', 'receipt_cumulative_gas_used', 'receipt_gas_used', 'block_number', 'value_cat', 'from_years_in_operation', 'to_years_in_operation']
Node DataFrame shape: (8819, 3)
Node feature columns: ['address', 'reg_year', 'max_value']
Response columns: ['from_scam', 'to_scam']


# Определяем модель

In [27]:
print("DataFrame columns:", df.columns.tolist())

DataFrame columns: ['hash', 'nonce', 'transaction_index', 'from_address', 'to_address', 'value', 'gas', 'gas_price', 'input', 'receipt_cumulative_gas_used', 'receipt_gas_used', 'block_timestamp', 'block_number', 'block_hash', 'from_scam', 'to_scam', 'from_category', 'to_category', 'any_scam', 'block_year', 'value_cat']


In [ ]:
import random
import numpy as np
import tensorflow as tf
import stellargraph as sg
from stellargraph import StellarDiGraph
from stellargraph.mapper import DirectedGraphSAGENodeGenerator
from stellargraph.layer import DirectedGraphSAGE
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy

# reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# 1) node_df и edge_df у вас уже готовы:
#    node_df = node_features_df.set_index('address')
#    edge_df = edge_df[['from_address','to_address'] + edge_feature_cols + response_cols]

# 2) Строим направленный StellarDiGraph, передаём только source/target
edges_sg = (
    edge_df[["from_address", "to_address"]]
    .rename(columns={"from_address": "source", "to_address": "target"})
)
graph = StellarDiGraph(nodes=node_df, edges=edges_sg)
print(graph.info())

# 3) Параметры сэмплинга: для каждого из 6 слоёв (4→32→16→16→16→4) указываем 
#    сколько in- и out-соседей мы будем выбирать. 
#    Здесь — по 10 соседей на уровне (можете подстроить под вашу задачу).
layer_sizes  = [4, 32, 16, 16, 16, 4]
in_samples   = [10] * len(layer_sizes)
out_samples  = [10] * len(layer_sizes)
batch_size   = 128

# 4) Инициализируем DirectedGraphSAGENodeGenerator
generator = DirectedGraphSAGENodeGenerator(
    graph,
    batch_size,
    in_samples=in_samples,
    out_samples=out_samples,
    seed=SEED
)

# 5) Создаём DirectedGraphSAGE с нужными слоями
dg_sage = DirectedGraphSAGE(
    layer_sizes=layer_sizes,
    generator=generator,
    bias=True,
    dropout=0.5,
)

# 6) Получаем Keras-тензоры
x_inp, x_out = dg_sage.in_out_tensors()

# 7) Финальный softmax-классификатор на 2 класса
predictions = Dense(units=2, activation="softmax", name="predictions")(x_out)
model = Model(inputs=x_inp, outputs=predictions, name="DirectedGraphSAGE")

# 8) Компиляция с Adam и categorical_crossentropy
model.compile(
    optimizer=Adam(),
    loss=categorical_crossentropy,
    metrics=["accuracy"],
)

# 9) Вывод структуры
model.summary()

StellarDiGraph: Directed multigraph
 Nodes: 8819, Edges: 10138

 Node types:
  default: [8819]
    Features: float32 vector, length 2
    Edge types: default-default->default

 Edge types:
    default-default->default: [10138]
        Weights: all 1 (default)
        Features: none


AttributeError: module 'stellargraph' has no attribute 'RGCN'

# Обучение модели

In [30]:
# %% [model training]
import pandas as pd
from sklearn.model_selection import train_test_split

# 1) Готовим метки узлов (any_scam):
from_label = df.groupby("from_address")["from_scam"].max()
to_label   = df.groupby("to_address")  ["to_scam"  ].max()
node_label = (
    from_label.reindex(node_df.index, fill_value=0).astype(int)
    | to_label.reindex(node_df.index, fill_value=0).astype(int)
)
node_targets = pd.get_dummies(node_label)  # колонки [0,1], one-hot

# 2) Сплит 50% / 25% / 25% с стратификацией
train_ids, rest_ids = train_test_split(
    node_targets.index,
    train_size=0.5,
    stratify=node_label,
    random_state=SEED,
)
val_ids, test_ids = train_test_split(
    rest_ids,
    train_size=0.5,
    stratify=node_label.reindex(rest_ids),
    random_state=SEED,
)
print("Sizes (train/val/test):", len(train_ids), len(val_ids), len(test_ids))

# 3) Генераторы flow для .fit()
train_gen = generator.flow(train_ids, node_targets.loc[train_ids], shuffle=True)
val_gen   = generator.flow(val_ids,   node_targets.loc[val_ids])

# %% [model training – debug debug]
print(">>> Старт ячейки обучения")        # самый первый проверочный принт

# Проверяем, что generator и train_gen определены
print("Generator:", generator)
print("Train sequence length:", len(train_gen))

# Пробуем взять первый батч через итератор
print(">>> Пробуем получить первый батч из train_gen")
try:
    batch0 = next(iter(train_gen))
    x0, y0 = batch0
    print("Успешно получили батч!")
    print("  • number of inputs tensors:", len(x0))
    print("  • shapes:", [arr.shape for arr in x0])
    print("  • labels shape:", y0.shape)
except Exception as e:
    print("Ошибка при получении батча:", e)

# Если мы дошли до сюда — generator рабочий, теперь фитим с прогресс-баром:
from tensorflow.keras.callbacks import Callback

class SimpleLogger(Callback):
    def on_epoch_begin(self, epoch, logs=None):
        print(f"\n>>> Epoch {epoch+1} start")
    def on_epoch_end(self, epoch, logs=None):
        print(f"    >>> Epoch {epoch+1} end; loss={logs.get('loss'):.4f}, acc={logs.get('accuracy'):.4f}")

print("\n>>> Начинаем обучение:")
history = model.fit(
    train_gen,
    epochs=20,
    steps_per_epoch=len(train_gen),
    validation_data=val_gen,
    validation_steps=len(val_gen),
    callbacks=[SimpleLogger()],
    verbose=1,
)
print(">>> fit() вернулся, обучение завершено")

# 5) Оценка на тесте
test_gen     = generator.flow(test_ids, node_targets.loc[test_ids])
test_loss, test_acc = model.evaluate(test_gen, verbose=2)
print(f"Test loss: {test_loss:.4f}, Test accuracy: {test_acc:.4f}")

>>> Старт ячейки обучения
Generator: <stellargraph.mapper.sampled_node_generators.DirectedGraphSAGENodeGenerator object at 0x402417790>
Train sequence length: 35
>>> Пробуем получить первый батч из train_gen


KeyboardInterrupt: 